In [16]:
# graph: undirected multicast figure-4 
# routing only

import cvxpy as cp

variables = cp.Variable(9)
fstar = cp.Variable()

def x(index):
    return variables[index - 1]

constraints = [
    x(3) + x(6) + x(8) <= 1,
    x(1) + x(5) + x(8) <= 1,
    x(2) + x(5) + x(9) <= 1,
    x(4) + x(6) + x(9) <= 1,
    x(1) + x(3) + x(7) <= 1,
    x(2) + x(4) + x(7) <= 1,
    x(1) + x(2) == x(3) + x(4),
    x(5) + x(6) + x(7) == x(1) + x(2),
    x(8) + x(9) == x(1) + x(2),
    fstar == x(1) + x(2)
]

objective = cp.Maximize(fstar)  
problem = cp.Problem(objective, constraints)
problem.solve()
print("Optimal Target Flow Rate (f*):", fstar.value)
for i in range(1, 10):
    print(f"x{i}:", x(i).value)


Optimal Target Flow Rate (f*): 0.7499999995001766
x1: 0.37499999975008813
x2: 0.37499999975008824
x3: 0.37499999975008813
x4: 0.3749999997500882
x5: 0.24999999983339213
x6: 0.2499999998333922
x7: 0.24999999983339208
x8: 0.3749999997500882
x9: 0.37499999975008813


In [33]:
# optimal solution

import cvxpy as cp
import numpy as np
import networkx as nx


inputnodes = [0,4]
inputedges = [(0,2),(2,3),(3,4),(4,0),(2,1),(1,4)]

non_member = []
source = [0,1,2,3]
recievers = [[1],[3],[4],[0]]

sessions = [0,1,2,3]


preG = nx.Graph()
preG.add_nodes_from(inputnodes)
preG.add_edges_from(inputedges)
directE = []
for n, nbrs in preG.adj.items():
    for nbr, eattr in nbrs.items():
        directE.append((n,nbr))


G = nx.DiGraph()
G.add_nodes_from(inputnodes)
G.add_edges_from(directE)

undirected_edges = set(inputedges)

vertices = set(np.arange(inputnodes[1]+1))
print(vertices)

edges = set(directE)
print(edges)


f_star = cp.Variable()  # Target flow rate


orientation_constraints = []
flow_constraints = []
equalrate_constraints = []


c =  {a : cp.Variable() for a in edges}

for a in edges:
    orientation_constraints += [c[a] >= 0]      #for every in 2E

for a1 in undirected_edges:         # for every in E
    a2 = (a1[1], a1[0]) 
    orientation_constraints += [ c[a1] + c[a2] == 1]



fia = {(i, a): cp.Variable() for a in edges for i in sessions}
fija = {(i, j, a): cp.Variable() for i in sessions for j in recievers[i] for a in edges}


for i in sessions: 
    for j in recievers[i]:
        for a in edges:
            flow_constraints += [fija[i,j,a] >= 0]
            flow_constraints += [fija[i,j,a] <= fia[i,a]]

for a in edges: 
    session_flow_edge = sum(fia[i,a] for i in sessions)
    flow_constraints += [session_flow_edge <= c[a]]



for i in sessions:
    for j in recievers[i]:
        non_member = vertices - {i} - {j}
        for vt in non_member:
            incoming_edges = [(u, v) for u, v in edges if v == int(vt)] 
            outcoming_edges = [(u, v) for u, v in edges if u == int(vt)] 
            incoming_flow = sum(fija[i, j, x] for x in incoming_edges)
            outcoming_flow = sum(fija[i, j, y] for y in outcoming_edges)
            flow_constraints += [incoming_flow == outcoming_flow]


        incoming_flow_source = sum(fija[i, j, x] for x in edges if x[1] == source[i])  
        
        outcoming_flow_reciever = sum(fija[i, j, x] for x in edges if x[0] == int(j)) 
        incoming_flow_reciever = sum(fija[i, j, x] for x in edges if x[1] == int(j))  

        flow_constraints += [incoming_flow_source == 0]
        flow_constraints += [outcoming_flow_reciever == 0]
        equalrate_constraints += [f_star == incoming_flow_reciever]




objective = cp.Maximize(f_star)    
constraints = orientation_constraints + flow_constraints + equalrate_constraints
problem = cp.Problem(objective, constraints)


problem.solve()

print("Optimal Target Flow Rate (f*):", f_star.value)
print(fia[0,(3,4)].value)

{0, 1, 2, 3, 4}
{(4, 0), (0, 4), (2, 1), (3, 4), (4, 3), (1, 2), (2, 0), (1, 4), (2, 3), (0, 2), (3, 2), (4, 1)}
Optimal Target Flow Rate (f*): 0.7499999999642175
5.6986065910210746e-12


In [ ]:
# graph: multiple unicast 2004 figure-3


import cvxpy as cp
import numpy as np
import networkx as nx


inputnodes = [0,5]
inputedges = [(0,2),(4,5),(1,3),(0,4),(4,1),(2,5),(5,3)]

non_member = [4,5]
source = [0,1]
recievers = [[3],[2]]

sessions = [0,1]



preG = nx.Graph()
preG.add_nodes_from(inputnodes)
preG.add_edges_from(inputedges)
directE = []
for n, nbrs in preG.adj.items():
    for nbr, eattr in nbrs.items():
        directE.append((n,nbr))


G = nx.DiGraph()
G.add_nodes_from(inputnodes)
G.add_edges_from(directE)

undirected_edges = set(inputedges)

vertices = set(np.arange(inputnodes[1]+1))
# print(vertices)

edges = set(directE)
# print(edges)


f_star = cp.Variable()  # Target flow rate


orientation_constraints = []
flow_constraints = []
equalrate_constraints = []


c =  {a : cp.Variable() for a in edges}

for a in edges:
    orientation_constraints += [c[a] >= 0]      #for every in 2E

for a1 in undirected_edges:         # for every in E
    a2 = (a1[1], a1[0]) 
    orientation_constraints += [ c[a1] + c[a2] == 1]



fia = {(i, a): cp.Variable() for a in edges for i in sessions}
fija = {(i, j, a): cp.Variable() for i in sessions for j in recievers[i] for a in edges}


for i in sessions: 
    for j in recievers[i]:
        for a in edges:
            flow_constraints += [fija[i,j,a] >= 0]
            flow_constraints += [fija[i,j,a] <= fia[i,a]]

for a in edges: 
    session_flow_edge = sum(fia[i,a] for i in sessions)
    flow_constraints += [session_flow_edge <= c[a]]



for i in sessions:
    for j in recievers[i]:
        non_member = vertices - {i} - {j}
        for vt in non_member:
            incoming_edges = [(u, v) for u, v in edges if v == int(vt)] 
            outcoming_edges = [(u, v) for u, v in edges if u == int(vt)] 
            incoming_flow = sum(fija[i, j, x] for x in incoming_edges)
            outcoming_flow = sum(fija[i, j, y] for y in outcoming_edges)
            flow_constraints += [incoming_flow == outcoming_flow]


        incoming_flow_source = sum(fija[i, j, x] for x in edges if x[1] == source[i])  
        
        outcoming_flow_reciever = sum(fija[i, j, x] for x in edges if x[0] == int(j)) 
        incoming_flow_reciever = sum(fija[i, j, x] for x in edges if x[1] == int(j))  

        flow_constraints += [incoming_flow_source == 0]
        flow_constraints += [outcoming_flow_reciever == 0]
        equalrate_constraints += [f_star == incoming_flow_reciever]




objective = cp.Maximize(f_star)    
constraints = orientation_constraints + flow_constraints + equalrate_constraints
problem = cp.Problem(objective, constraints)


problem.solve()

print("Optimal Target Flow Rate (f*):", f_star.value)

In [38]:
# graph: example 3 in my notes
# routing only

import cvxpy as cp

variables = cp.Variable(10)
fstar = cp.Variable()

def x(index):
    return variables[index - 1]

# 8 edges
constraints = [
    x(2) + x(4) + x(9) <= 1,
    x(1) + x(4) + x(8) <= 1,
    x(3) + x(8) + x(9) <= 1,
    x(2) + x(5) + x(10) <= 1,
    x(1) + x(5) + x(7) <= 1,
    x(3) + x(7) + x(10) <= 1,
    x(6) + x(9) + x(10) <= 1,
    x(6) + x(7) + x(8) <= 1,

    x(1) + x(2) + x(3) == x(4) + x(5) + x(6),
    x(1) + x(2) + x(3) == x(7) + x(8) + x(9) + x(10),
    fstar == x(1) + x(2) + x(3)
]

objective = cp.Maximize(fstar)  
problem = cp.Problem(objective, constraints)
problem.solve()
print("Optimal Target Flow Rate (f*):", fstar.value)
for i in range(1, 11):
    print(f"x{i}:", x(i).value)

Optimal Target Flow Rate (f*): 1.1428571423306586
x1: 0.35714285697833076
x2: 0.3571428569783307
x3: 0.42857142837399687
x4: 0.3571428569783307
x5: 0.35714285697833076
x6: 0.42857142837399687
x7: 0.28571428558266454
x8: 0.28571428558266465
x9: 0.28571428558266454
x10: 0.28571428558266465


In [39]:
# graph: multiple unicast 2004 figure-3


import cvxpy as cp
import numpy as np
import networkx as nx


inputnodes = [0,5]
inputedges = [(0,1),(0,4),(0,2),(3,1),(3,4),(3,2),(5,1),(5,4)]

non_member = []
source = [0,1,2]
recievers = [[3],[4],[5]]

sessions = [0,1,2]



preG = nx.Graph()
preG.add_nodes_from(inputnodes)
preG.add_edges_from(inputedges)
directE = []
for n, nbrs in preG.adj.items():
    for nbr, eattr in nbrs.items():
        directE.append((n,nbr))


G = nx.DiGraph()
G.add_nodes_from(inputnodes)
G.add_edges_from(directE)

undirected_edges = set(inputedges)

vertices = set(np.arange(inputnodes[1]+1))
# print(vertices)

edges = set(directE)
# print(edges)


f_star = cp.Variable()  # Target flow rate


orientation_constraints = []
flow_constraints = []
equalrate_constraints = []


c =  {a : cp.Variable() for a in edges}

for a in edges:
    orientation_constraints += [c[a] >= 0]      #for every in 2E

for a1 in undirected_edges:         # for every in E
    a2 = (a1[1], a1[0]) 
    orientation_constraints += [ c[a1] + c[a2] == 1]



fia = {(i, a): cp.Variable() for a in edges for i in sessions}
fija = {(i, j, a): cp.Variable() for i in sessions for j in recievers[i] for a in edges}


for i in sessions: 
    for j in recievers[i]:
        for a in edges:
            flow_constraints += [fija[i,j,a] >= 0]
            flow_constraints += [fija[i,j,a] <= fia[i,a]]

for a in edges: 
    session_flow_edge = sum(fia[i,a] for i in sessions)
    flow_constraints += [session_flow_edge <= c[a]]



for i in sessions:
    for j in recievers[i]:
        non_member = vertices - {i} - {j}
        for vt in non_member:
            incoming_edges = [(u, v) for u, v in edges if v == int(vt)] 
            outcoming_edges = [(u, v) for u, v in edges if u == int(vt)] 
            incoming_flow = sum(fija[i, j, x] for x in incoming_edges)
            outcoming_flow = sum(fija[i, j, y] for y in outcoming_edges)
            flow_constraints += [incoming_flow == outcoming_flow]


        incoming_flow_source = sum(fija[i, j, x] for x in edges if x[1] == source[i])  
        
        outcoming_flow_reciever = sum(fija[i, j, x] for x in edges if x[0] == int(j)) 
        incoming_flow_reciever = sum(fija[i, j, x] for x in edges if x[1] == int(j))  

        flow_constraints += [incoming_flow_source == 0]
        flow_constraints += [outcoming_flow_reciever == 0]
        equalrate_constraints += [f_star == incoming_flow_reciever]




objective = cp.Maximize(f_star)    
constraints = orientation_constraints + flow_constraints + equalrate_constraints
problem = cp.Problem(objective, constraints)


problem.solve()

print("Optimal Target Flow Rate (f*):", f_star.value)

Optimal Target Flow Rate (f*): 1.1428571428543588


In [5]:
import cvxpy as cp
import networkx as nx

# Example graph setup
inputnodes = [0, 5]
inputedges = [(0, 1), (0, 4), (0, 2), (3, 1), (3, 4), (3, 2), (5, 1), (5, 4)]
G = nx.DiGraph()
G.add_nodes_from(inputnodes)
G.add_edges_from(inputedges)

# Sessions: source node and destination nodes
sessions = {
    0: {'source': 0, 'destinations': [3, 4]},
    1: {'source': 5, 'destinations': [1, 2]}
}

# Flow variables for each edge in each session
f = {(s, e): cp.Variable(nonneg=True) for s in sessions for e in G.edges}

# Objective: Maximize the minimum flow across all sessions
# Objective: Maximize the minimum flow across all sessions
objective = cp.Maximize(cp.minimum([cp.sum([f[s, e] for e in G.out_edges(sessions[s]['source'])]) for s in sessions]))

# Constraints
constraints = []

# Flow conservation for each session
for s in sessions:
    for node in G.nodes:
        if node == sessions[s]['source']:
            # Total outgoing flow from source
            constraints.append(cp.sum(f[s, e] for e in G.out_edges(node)) == cp.sum(f[s, e] for e in G.in_edges(node)))
        elif node in sessions[s]['destinations']:
            # Flow into destinations
            constraints.append(cp.sum(f[s, e] for e in G.in_edges(node)) == cp.sum(f[s, e] for e in G.out_edges(node)))
        else:
            # Flow conservation at intermediate nodes
            constraints.append(cp.sum(f[s, e] for e in G.in_edges(node)) == cp.sum(f[s, e] for e in G.out_edges(node)))

# Capacity constraints (assuming each edge has a capacity of 1)
for e in G.edges:
    constraints.append(cp.sum(f[s, e] for s in sessions) <= 1)

# Problem
problem = cp.Problem(objective, constraints)

# Solve
problem.solve()

# Output
print("Optimal Routing Throughput:", problem.value)
for s in sessions:
    for e in G.edges:
        print(f"Session {s}, Flow on edge {e}: {f[s, e].value}")


TypeError: float() argument must be a string or a real number, not 'generator'